In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('data/graduate_admission.csv')

In [3]:
data.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [4]:
dummy = pd.get_dummies(data['rank'])

In [5]:
data = pd.concat([data,dummy],axis=1)

In [6]:
data = data.drop("rank",axis=1)

In [7]:
for field in ["gre","gpa"]:
    mean,std = data[field].mean(),data[field].std()
    data[field] = (data[field]-mean)/std

In [8]:
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

In [9]:
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [10]:
features.head()

,gre,gpa,1,2,3,4
122,-0.586063,-1.418674,0,0,1,0
242,0.799020,-1.129631,0,0,1,0
213,0.452749,-0.026014,0,0,1,0
16,1.664697,1.261539,0,0,0,1
91,1.145291,0.657178,1,0,0,0


In [11]:
targets.head()

122    0
242    1
213    0
16     0
91     1
Name: admit, dtype: int64

In [13]:
#Activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [26]:
weights.shape

(6,)

In [47]:
n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.01

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(weights,x))

        # TODO: Calculate the error
        error = (y - output)

        # TODO: Calculate the error term
        error_term = error*(output*(1-output))

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term*x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate*del_w

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2711287709809773
Train loss:  0.20009244020167583
Train loss:  0.19996582166604635
Train loss:  0.1999609809592638
Train loss:  0.1999607370919641
Train loss:  0.19996072418224048
Train loss:  0.19996072349134103
Train loss:  0.19996072345427301
Train loss:  0.19996072345228347
Train loss:  0.19996072345217647
Prediction accuracy: 0.800


In [50]:
#Two layer

# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x,weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output,weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = (y-output)

        # TODO: Calculate error term for the output unit
        output_error_term = error*(output*(1-output))

        ## propagate errors to hidden layer
       ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)

        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)

        # TODO: Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.3622119361830061
Train loss:  0.3593857686325661
Train loss:  0.3565810071410853
Train loss:  0.3537992165909672
Train loss:  0.3510419129469353
Train loss:  0.3483105588982085
Train loss:  0.34560655983074634
Train loss:  0.3429312601544241
Train loss:  0.3402859400058011
Train loss:  0.3376718123425667
Prediction accuracy: 0.200
